In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [ ]:
from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import mimetypes

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive']


def upload_model(path, filename):
    """Insert new file.
    Returns : Id's of the file uploaded

    Load pre-authorized user credentials from the environment.
    TODO(developer) - See https://developers.google.com/identity
    for guides on implementing OAuth2 for the application.
    """
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    try:
        # create drive api client
        service = build('drive', 'v3', credentials=creds)
        
        mimetype = mimetypes.guess_type(filename)
        
        real_folder_id = '1QKzZFTLCc_OJbOLIwKsnsO5zxGRyCc6v'

        file_metadata = {'name': filename, 'parents': [real_folder_id]}
        media = MediaFileUpload(os.path.join(path, filename),
                                mimetype=mimetype[0], resumable=True)
        # pylint: disable=maybe-no-member
        file = service.files().create(body=file_metadata, media_body=media,
                                      fields='id')
        
        response = None
        while response is None:
            status, response = file.next_chunk()
            if status:
                    print ("Uploaded %d%%." % int(status.progress() * 100))

        if file:
            print(filename + " uploaded successfully")    

    except HttpError as error:
        print(F'An error occurred: {error}')
        file = None



In [ ]:
model = '2022-10-23T09-51-56_milenka_v1.5_18_training_images_2020_max_training_steps_Rakhee_Thakrar_token_woman_class_word.ckpt'
model = '2022-10-23T11-35-13_luis_v1.5_20_training_images_2020_max_training_steps_Charlie_Cox_token_man_class_word.ckpt'

model = '2022-10-23T12-45-20_alberto_v1.5_12_training_images_2020_max_training_steps_Tom_McCarthy_token_man_class_word.ckpt'
model = '2022-10-23T15-04-45_mama_v1.5_12_training_images_2020_max_training_steps_Carrie_Fisher_token_woman_class_word.ckpt'
model = '2022-10-23T13-55-33_papa_v1.5_12_training_images_2020_max_training_steps_Tom_McCarthy_token_man_class_word.ckpt'
model = '2022-10-23T16-28-33_morgane_v1.5_7_training_images_2020_max_training_steps_Chelsea_Field_token_woman_class_word.ckpt'
model = '2022-10-23T17-36-28_daniel_v1.5_7_training_images_2020_max_training_steps_Andrzej_Bartkowiak_token_man_class_word.ckpt'
model = '2022-10-23T18-54-24_urbana_v1.5_8_training_images_1500_max_training_steps_Mira_Nair_token_woman_class_word.ckpt'
model = '2022-10-23T19-57-57_kali_v1.5_5_training_images_1500_max_training_steps_Queen_Latifah_token_woman_class_word.ckpt'

upload_model('trained_models', model)